In [3]:
from utils.trainer import ClassifierTrainer

In [4]:
from utils.nn import create_mlp_layers
import torch


clf = torch.nn.Sequential(*create_mlp_layers(784, [300, 100], 10))

In [5]:
from sklearn.datasets import fetch_openml


mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = mnist['data'], mnist['target']

In [2]:
class NamedDataset(torch.utils.data.Dataset):
    def __init__(self, names: list[str], data: list[torch.Tensor]):
        self.names = names
        self.data = data
        
    def __len__(self) -> int:
        return len(self.data[0])
    
    def __getitem__(self, idx) -> dict[str, torch.Tensor]:
        return {name: data[idx] for name, data in zip(self.names, self.data)}


def create_generator(dataset: torch.utils.data.Dataset, batch_size: int = 128, shuffle: bool = True, drop_last: bool = True, **kwargs):
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, **kwargs)
    while True:
        yield from loader

In [7]:
train_generator = create_generator(NamedDataset(['x', 'y'], [torch.tensor(X.values) / 255.0, y.values.astype(int)]))

In [6]:
trainer = ClassifierTrainer(clf)

trainer.train(train_generator)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


100%|██████████| 5000/5000 [01:22<00:00, 60.35it/s]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss/train,█▆▄▅▃▂▃▂▂▁▂▂▂▁▁▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss/train,0.00999


In [3]:
from utils.bmds import BMDS, BMDSTrainer
from utils.bmds_data import DefaultBMDSDataset
import torch

x = torch.randn(100, 2)

dist = (x[:, None] - x).pow(2).sum(2).pow(0.5)

bmds = BMDS(100, 100)
bmds_trainer = BMDSTrainer(bmds)
bmds_dataset = DefaultBMDSDataset(dist)

bmds_trainer.train(create_generator(bmds_dataset))

wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


  0%|          | 0/5000 [01:04<?, ?it/s]


KeyboardInterrupt: 